In [6]:
 !nvcc --version
 %env OMP_NUM_THREADS=3

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
env: OMP_NUM_THREADS=3


In [7]:
 %%writefile bfs1.cpp
 #include <iostream>
 #include <queue>
 #include <omp.h>
 using namespace std;
 const int MAX = 1000;
 int graph[MAX][MAX], visited[MAX];
 void bfs(int start, int n) {
  queue<int> q;
  visited[start] = 1;
  q.push(start);
  while(!q.empty()) {
    int curr = q.front();
    q.pop();
    #pragma omp parallel for shared(graph, visited, q) schedule(dynamic)
    for(int i=0; i<n; i++) {
      if(graph[curr][i] && !visited[i]) {
        visited[i] = 1;
        q.push(i);
      }
    }
  }
 }
 int main() {
  int n, start;
  cout << "Enter number of vertices: ";
  cin >> n;
  cout << "Enter adjacency matrix:\n";
  for(int i=0; i<n; i++) {
    for(int j=0; j<n; j++) {
      cin >> graph[i][j];
    }
  }
  cout << "Enter starting vertex: ";
  cin >> start;
  #pragma omp parallel num_threads(4)
  {
    bfs(start, n);
  }
  cout << "BFS traversal: ";
  for(int i=0; i<n; i++) {
    if(visited[i])
    cout << i << " ";
  }
  cout << endl;
  return 0;
 }

Writing bfs1.cpp


In [8]:
!g++ bfs1.cpp -o bfs1 -fopenmp

In [9]:
 !./bfs1

Enter number of vertices: 5
Enter adjacency matrix:
0 1 1 0 0
1 0 0 1 0
1 0 0 1 1
0 1 1 0 1
0 0 1 1 0
Enter starting vertex: 0
BFS traversal: 0 1 2 3 4 
